# Hierarchical latent variational autoencoders
From [Gulrajani+] PixelVAE: A Latent Variable Model for Natural Images
* $p(x,z)=p(x|z)p(z)$
    * $p(z)=p(z_1|z_2)...p(z_{n-1}|z_n)p(z_n)$
* $q(z|x)=q(z_n|x)...q(z_1|x)$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from progressbar import ProgressBar
import time,os

In [ ]:
from lasagne.layers import InputLayer,DenseLayer
from lasagne.nonlinearities import rectify,linear,softplus,sigmoid
from lasagne.updates import adam

In [ ]:
from Tars.models import VAE
from Tars.distributions import Gaussian,Bernoulli,MultiDistributions,MultiPriorDistributions
from Tars.load_data import mnist

In [ ]:
load,plot = mnist('../datasets/')
train_x,_,_,_,test_x,_ = load(test=True)

activation = rectify
seed = 1234
np.random.seed(seed)

n_epoch = 100
n_batch = 100

optimizer = adam
optimizer_params={"learning_rate":0.001, "epsilon":1e-4}
clip_grad=1
max_norm_constraint=5

In [ ]:
n_units = [784,64,32,18]

In [ ]:
q = []
for i, n_input, n_output in zip(reversed(range(len(n_units)-1)), n_units[:-1], n_units[1:]):
    print n_input, n_output
    x = InputLayer((None,n_input))
    q_0  = DenseLayer(x,num_units=512,nonlinearity=activation)
    q_1  = DenseLayer(q_0,num_units=512,nonlinearity=activation)
    q_mean = DenseLayer(q_1,num_units=n_output,nonlinearity=linear)
    q_var = DenseLayer(q_1,num_units=n_output,nonlinearity=softplus)
    _q = Gaussian(q_mean,q_var,given=[x]) #p(z|x) or p(z|z)
    q.append(_q)

q = MultiDistributions(q)

In [ ]:
prior = []
for i, n_input, n_output in zip(reversed(range(len(n_units)-2)), reversed(n_units[2:]), reversed(n_units[1:-1])):
    print n_input,n_output
    z = InputLayer((None,n_input))
    p_0  = DenseLayer(z,num_units=512,nonlinearity=activation)
    p_1  = DenseLayer(p_0,num_units=512,nonlinearity=activation)
    p_mean = DenseLayer(p_1,num_units=n_output,nonlinearity=linear)
    p_var = DenseLayer(p_1,num_units=n_output,nonlinearity=softplus)
    _p = Gaussian(p_mean,p_var,given=[z]) #p(z|z) 
    prior.append(_p)

prior = MultiPriorDistributions(prior)

z = InputLayer((None,n_units[1]))
p_0  = DenseLayer(z,num_units=512,nonlinearity=activation)
p_1  = DenseLayer(p_0,num_units=512,nonlinearity=activation)
p_mean = DenseLayer(p_1,num_units=n_units[0],nonlinearity=sigmoid)
p = Bernoulli(p_mean,given=[z]) #p(x|z)

In [ ]:
model = VAE(q, p, prior,
            n_batch=n_batch, optimizer=adam,
            optimizer_params=optimizer_params,
            train_iw=False, test_iw=True,
            clip_grad=clip_grad, max_norm_constraint=max_norm_constraint)

In [ ]:
n_sample = 100
sample_z  = np.random.standard_normal((n_batch, n_units[-1])).astype(np.float32)

def plot_image(t,i):
    _sample_z = prior.np_sample_given_x(sample_z)
    sample_x = p.np_sample_mean_given_x(_sample_z)
    fig = plt.figure(figsize=(10,10))
    X,cmap = plot(sample_x[:n_sample])

    for j,x in enumerate(X):
            ax = fig.add_subplot(10, 10, j + 1)
            ax.imshow(x,cmap)
            ax.axis('off')

    plt.savefig('../plot/%d/%04d.jpg'%(t,i))
    plt.close()

In [ ]:
t = int(time.time())
os.mkdir('../plot/%d' % t)

pbar = ProgressBar(maxval=n_epoch).start()
for i in range(1, n_epoch+1):
    np.random.shuffle(train_x)
    lowerbound_train = model.train([train_x])

    if (i%10 == 0) or (i == 1):
        log_likelihood_test = model.test([test_x])
        lw = "epoch = %d, lower bound (train) = %lf (%lf %lf) lower bound (test) = %lf\n" %(i,sum(lowerbound_train),lowerbound_train[0],lowerbound_train[1],np.mean(log_likelihood_test))
        f = open("../plot/%d/temp.txt" % t, "a")
        f.write(lw)
        f.close()
        print lw
        plot_image(t,i)
        
    pbar.update(i)